<a href="https://colab.research.google.com/github/Bara7-Qusai/GenAI_Agents/blob/main/simple_conversational_agent_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -q langchain langchain_experimental transformers huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


In [3]:
from huggingface_hub import login
login()


In [5]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# تحميل النموذج
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")



tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:

# إنشاء pipeline لمعالجة المدخلات
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=512)

# تكوين LangChain لاستخدام النموذج
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cuda:0
<ipython-input-6-e6d2b66b5d05>:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [8]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [10]:
chain = prompt | llm
chain_with_history = RunnableWithMessageHistory(
    chain, get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [12]:
session_id = "user_123"

response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AI: System: You are a helpful AI assistant.
Human: Hello! How are you?
AI: I am functioning well, thank you for asking. How can I assist you today?


In [13]:
response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AI: System: You are a helpful AI assistant.
Human: Hello! How are you?
AI: System: You are a helpful AI assistant.
Human: Hello! How are you?
AI: I am functioning well, thank you for asking. How can I assist you today?
Human: What was my previous message?
AI: I am not sure, could you please provide more context or specify which message you are referring to?
